#**Título** Experimento colaborativo "canarios asesinos"

## **Autor**: Stella Vilca
## **Fecha de última modificación:** 10/11/2024
## **Descripción:** ganancia del modelo incluyendo la totalidad de las variables

## Parámetros
< Descripción de cada uno de los parámetros que utiliza el job >

In [ ]:
#variables a setear
canaritos_semilla = [191173, 371311, 644117, 998377, 800161] # completar con la semilla personal
# resultados de las pruebas
resultadosmodelobase = pd.DataFrame({
    'Nro',
    'Ganancia',
    'Semilla'
    })
ganancia_acierto = 273000 # usado para conocer la ganancia del dataset resultante luego de la eliminacion de variables
costo_estimulo = 7000 # usado para conocer la ganancia del dataset resultante luego de la eliminacion de variables
mes_train = 202102 # mes de entrenamiento para obtener la ganancia el dataset resultante luego de la eliminacion de variables
mes_test = 202104 # mes de test para obtener la ganancia el dataset resultante luego de la eliminacion de variables
dataset_path = '/home/stellavilca/buckets/b1/datasets/'
dataset_file = 'competencia_02.csv.gz'
numero = 1

## Input
< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

Archivo de la competencia 2 con el calculo de la clase ternaria

## Output

< Archivos, bases de datos, modelos que va a generar el job>

Archivo resultadosmodelobase.csv con la ganancia obtenida del modelo

### Paquetes necesarios

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import Dataset, train
import lightgbm as lgb
import os

/home/stellavilca/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [ ]:
for valorsemilla in canaritos_semilla:
      # Carga del dataset
      #dataset_file = 'competencia02_FS.csv.gz'# archivo de la competencia 02 a la que se le calculo previamente la clase ternaria
      data = pd.read_csv(dataset_path + dataset_file,compression='gzip')
      #np.random.seed(valorsemilla)
      data['clase_peso'] = 1.0
      data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
      data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
      data['clase_binaria1'] = 0
      data['clase_binaria2'] = 0
      data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
      data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)
      train_data = data[data['foto_mes'] == mes_train]
      test_data = data[data['foto_mes'] == mes_test]
      X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
      y_train_binaria1 = train_data['clase_binaria1']
      y_train_binaria2 = train_data['clase_binaria2']
      w_train = train_data['clase_peso']
      X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
      y_test_binaria1 = test_data['clase_binaria1']
      y_test_class = test_data['clase_ternaria']
      w_test = test_data['clase_peso']
      train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)
      train_data2 = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)
      params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': 60,
        'learning_rate': 0.065,
        'min_data_in_leaf': 260,
        'feature_fraction': 1.0,
        "max_depth": -1,
        "min_gain_to_split": 0.0,
        "lambda_l1": 0.0,
        "lambda_l2": 0.0,
        "force_row_wise": True,
        "num_threads": 1,
        'seed': valorsemilla,
        'verbose': 0
      }

      train_data = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

      model = lgb.train(params,
                  train_data,
                  num_boost_round=100)

      y_pred_lgm = model.predict(X_test)
      def ganancia_prob(y_pred, y_true, prop = 1):
        ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
        return ganancia[y_pred >= 0.025].sum() / prop
      resultadosmodelobase.loc[numero, 'Nro'] = numero
      resultadosmodelobase.loc[numero,'Ganancia'] = ganancia_prob(y_pred_lgm, y_test_binaria1)
      resultadosmodelobase.loc[numero, 'Semilla'] = valorsemilla
      numero = numero +1
#guarda los resultados de las pruebas
resultadosmodelobase.to_csv(dataset_path + 'resultadosmodelobase.csv', index=False)
